In [40]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier

In [2]:
dfMasterTable = pd.read_csv('dfMasterTable.csv')

In [3]:
dfMasterTable.head()

,msno,is_churn,date_pred,bd,gender,city_1,city_4,city_5,city_13,city_15,...,num_unq_sum,num_unq_mean,num_unq_std,msno_count_y,nbr_logins_monthly_mean,nbr_logins_monthly_max,nbr_logins_monthly_min,nbr_logins_monthly_std,nbr_logins_monthly_count,last_login_days
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,201702,36.0,0.0,0.0,0.0,0.0,0.0,0.0,...,394.0,19.700000,16.012807,20.0,8.666667,18.0,2.0,8.326664,3.0,3.0
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,201702,38.0,1.0,0.0,0.0,0.0,0.0,0.0,...,7229.0,13.901923,14.006454,520.0,20.840000,31.0,1.0,9.485427,25.0,33.0
2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1,201702,27.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11533.0,48.662447,35.829472,237.0,19.750000,30.0,4.0,8.945542,12.0,1.0
3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1,201702,23.0,0.0,0.0,0.0,0.0,1.0,0.0,...,17943.0,25.130252,24.978054,714.0,28.269231,31.0,21.0,3.244166,26.0,2.0
4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1,201702,27.0,1.0,0.0,0.0,0.0,0.0,0.0,...,71565.0,97.500000,45.000447,734.0,29.153846,31.0,21.0,2.460769,26.0,2.0


In [47]:
dfMasterTable.fillna(0, inplace=True)

In [48]:
train_set = dfMasterTable[dfMasterTable.date_pred < 201703]
test_set = dfMasterTable[dfMasterTable.date_pred == 201703]

In [49]:
train_set.isnull().mean()

msno                        0.0
is_churn                    0.0
date_pred                   0.0
bd                          0.0
gender                      0.0
city_1                      0.0
city_4                      0.0
city_5                      0.0
city_13                     0.0
city_15                     0.0
city_22                     0.0
city_other                  0.0
registered_via_3            0.0
registered_via_4            0.0
registered_via_7            0.0
registered_via_9            0.0
registered_via_11           0.0
registered_via_13           0.0
registered_via_other        0.0
tenure_days                 0.0
actual_amount_paid_min      0.0
actual_amount_paid_max      0.0
actual_amount_paid_mean     0.0
actual_amount_paid_std      0.0
actual_amount_paid_sum      0.0
is_auto_renew_mean          0.0
is_auto_renew_sum           0.0
is_cancel_sum               0.0
is_cancel_mean              0.0
msno_count_x                0.0
diff_dates_min              0.0
diff_dat

In [50]:
train_set.shape

(992931, 50)

In [51]:
test_set.shape

(970960, 50)

In [52]:
X_train, y_train = train_set.drop(['msno', 'is_churn', 'date_pred'], 1).values, train_set.is_churn.values
X_test, y_test = test_set.drop(['msno', 'is_churn', 'date_pred'], 1).values, test_set.is_churn.values

In [53]:
clf = XGBClassifier(n_jobs=-1)

In [54]:
clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=-1, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [55]:
y_pred = clf.predict_proba(X_test)[:, 1]

In [56]:
roc_auc_score(y_test, y_pred)

0.8383135342038617

In [57]:
log_loss(y_test, y_pred)

0.22475174649606028

In [58]:
rf = RandomForestClassifier(n_estimators=400, n_jobs=-1)

In [59]:
rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=400, n_jobs=-1)

In [62]:
y_pred = rf.predict_proba(X_test)[:, 1]

In [63]:
log_loss(y_test, y_pred)

0.24658377413898053

In [64]:
roc_auc_score(y_test, y_pred)

0.8401701599788997